## Container Runtime HPO Example
This example notebook demonstrates how to use the container runtime HPO API to train a simple XGBoost model with Bayesian optimization, random search, and grid search. It highlights both single-node and multi-node HPO—powered by the same API—with multi-node support enabled through an optional scale_cluster call. The notebook also shows how to retrieve and analyze HPO results via the API.

In [1]:
import xgboost as xgb
from snowflake.ml.data.data_connector import DataConnector
from snowflake.ml.modeling import tune
from snowflake.ml.modeling.tune import get_tuner_context
from sklearn import datasets
from snowflake.ml.modeling.tune.search import BayesOpt, RandomSearch, GridSearch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from snowflake.snowpark.context import get_active_session

### Data Ingestion & Define Training Function

In [2]:
######### STEP 0: FOLLOWING CODE SHOULD ALREADY BE AUTO-GENERATED IN SNOWFLAKE NOTEBOOK ##########

session = get_active_session()
    
######### STEP 1: GENERATE ARTIFICIAL TRAINING DATA FOR ILLUSTRATION PURPOSES ##########
X, y = datasets.load_digits(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train = X_train.assign(target=y_train).reset_index(drop=True)
X_test = X_test.assign(target=y_test).reset_index(drop=True)
dataset_map = {
    "train": DataConnector.from_dataframe(session.create_dataframe(X_train)),
    "test": DataConnector.from_dataframe(session.create_dataframe(X_test)),
}

######### STEP 2: DEFINE TRAINING FUNCTION ##########
def train_func():
    tuner_context = get_tuner_context()
    config = tuner_context.get_hyper_params()
    dm = tuner_context.get_dataset_map()
    train_df = dm["train"].to_pandas()
    test_df = dm["test"].to_pandas()
    train_labels = train_df['"target"']
    train_features = train_df.drop(columns=['"target"'])
    test_labels = test_df['"target"']
    test_features = test_df.drop(columns=['"target"'])
    model = xgb.XGBClassifier(
        **{k: int(v) if k != "learning_rate" else v for k, v in config.items()},
        random_state=42,
    )
    model.fit(train_features, train_labels)
    accuracy = accuracy_score(
        test_labels, model.predict(test_features)
    )
    tuner_context.report(metrics={"accuracy": accuracy}, model=model)

18-Feb-25 17:51:48 - MLRC - WARNING - SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 
18-Feb-25 17:51:48 - MLRC - INFO - Reading /Users/shchen/.snowsql/config for connection parameters defined as connections
/Users/shchen/snowflake/mlruntimes/env_service/lib/python3.11/site-packages/snowflake/connector/config_manager.py:351: UserWarning: Bad owner or permissions on /Users/shchen/.snowflake/connections.toml.
 * To change owner, run `chown $USER "/Users/shchen/.snowflake/connections.toml"`.
 * To restrict permissions, run `chmod 0600 "/Users/shchen/.snowflake/connections.toml"`.

  warn(f"Bad owner or permissions on {str(filep)}{chmod_message}")
18-Feb-25 17:51:48 - MLRC - INFO - Snowflake Connector for Python Version: 3.12.2, Python Version: 3.11.8, Platform: macOS-15.3.1-arm64-arm-64bit
18-Feb-25 17:51:48 - MLRC - INFO - Connecting to GLOBAL Snowflake domain
18-Feb-25 17:51:48 - MLRC - INFO - This connection is in OCSP Fail Open Mode. TLS Certif

### [OPTIONAL STEP] Scale Up Cluster To Enable Multi-Node HPO

In [ ]:
from snowflake.ml.runtime_cluster import scale_cluster

scale_cluster(2) # scale up from single node to two nodes

### Bayesian Optimization Search

In [ ]:
######### STEP 3: START HPO RUN With Bayes Opt Search ##########
tuner = tune.Tuner(
    train_func=train_func,
    search_space={
        "n_estimators": tune.uniform(50, 200),
        "max_depth": tune.uniform(3, 10),
        "learning_rate": tune.uniform(0.01, 0.3),
    },
    tuner_config=tune.TunerConfig(
        metric="accuracy",
        mode="max",
        search_alg=BayesOpt(),
        num_trials=3, # Increase num_trials for broader exploration and potentially better model performance
    ),
)

tuner_results = tuner.run(dataset_map=dataset_map)

In [4]:
######### STEP 4: EVALUATE THE HPO RUN RESULT ##########

tuner_results.best_result

,accuracy,should_checkpoint,trial_id,time_total_s,config/learning_rate,config/n_estimators,config/max_depth
0,1.0,True,d9a27959,7.045038,0.118617,159.799091,9.655


In [6]:
tuner_results.best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.11861663879080009,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=159, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

### Random Search

In [7]:
######### START HPO RUN With Random Search ##########

tuner = tune.Tuner(
    train_func=train_func,
    search_space={
        "n_estimators": tune.uniform(50, 200),
        "max_depth": tune.uniform(3, 10),
        "learning_rate": tune.uniform(0.01, 0.3),
    },
    tuner_config=tune.TunerConfig(
        metric="accuracy",
        mode="max",
        search_alg=RandomSearch(),
        num_trials=3,  # Increase num_trials for broader exploration and potentially better model performance
    ),
)

tuner_results = tuner.run(dataset_map=dataset_map)

18-Feb-25 17:52:19 - MLRC - INFO - Reading /Users/shchen/.snowsql/config for connection parameters defined as connections
18-Feb-25 17:52:19 - MLRC - INFO - Reading /Users/shchen/.snowsql/config for connection parameters defined as connections
18-Feb-25 17:52:19 - MLRC - INFO - Reading /Users/shchen/.snowsql/config for connection parameters defined as connections
18-Feb-25 17:52:20 - MLRC - INFO - Reading /Users/shchen/.snowsql/config for connection parameters defined as connections


2025-02-18 17:52:20,154	INFO job_manager.py:528 -- Runtime env is setting up.

Trial status: 1 PENDING
Current time: 2025-02-18 17:52:23. Total running time: 0s

(ReadResultSetDataSource pid=55180) /Users/shchen/snowflake/mlruntimes/env_service/lib/python3.11/site-packages/snowflake/connector/config_manager.py:351: UserWarning: Bad owner or permissions on /Users/shchen/.snowflake/connections.toml.

(ReadResultSetDataSource pid=55180)  * To change owner, run `chown $USER "/Users/shchen/.snowflake/connections.toml"`.
(ReadResultSetDataSource pid=55180)  * To restrict permissions, run `chmod 0600 "/Users/shchen/.snowflake/connections.toml"`.
(ReadResultSetDataSource pid=55180) 

(ReadResultSetDataSource pid=55180)   warn(f"Bad owner or permissions on {str(filep)}{chmod_message}")

(ReadResultSetDataSource pid=55179) /Users/shchen/snowflake/mlruntimes/env_service/lib/python3.11/site-packages/snowflake/connector/config_manager.py:351: UserWarning: Bad owner or permissions on /Users/shchen/.

In [8]:
tuner_results.results

,accuracy,should_checkpoint,trial_id,time_total_s,config/learning_rate,config/n_estimators,config/max_depth
0,1.0,True,2901a_00000,5.854237,0.240366,111.299604,3.397457
1,1.0,True,2901a_00001,4.121719,0.054205,196.605421,7.230751


### Grid Search

In [9]:
######### START HPO RUN With Grid Search ##########

tuner = tune.Tuner(
    train_func=train_func,
    search_space = {
        "n_estimators": [50, 51],
        "max_depth": [4,5],
        "learning_rate": [0.01, 0.03]
    },
    tuner_config=tune.TunerConfig(
        metric="accuracy",
        mode="max",
        search_alg=GridSearch(),
        max_concurrent_trials=2,  # (Optional) Maximum number of trials to run concurrently. If not set, defaults to the number of nodes in the cluster.
        resource_per_trial={"CPU": 1},   # (Optional) Pre-configured for reliability; modification is rarely necessary.
    ),
)

tuner_results = tuner.run(dataset_map=dataset_map)

18-Feb-25 17:52:39 - MLRC - INFO - Reading /Users/shchen/.snowsql/config for connection parameters defined as connections
18-Feb-25 17:52:39 - MLRC - INFO - Reading /Users/shchen/.snowsql/config for connection parameters defined as connections
18-Feb-25 17:52:39 - MLRC - INFO - Reading /Users/shchen/.snowsql/config for connection parameters defined as connections
18-Feb-25 17:52:39 - MLRC - INFO - Reading /Users/shchen/.snowsql/config for connection parameters defined as connections


2025-02-18 17:52:40,265	INFO job_manager.py:528 -- Runtime env is setting up.

Trial status: 1 PENDING
Current time: 2025-02-18 17:52:43. Total running time: 0s

(ReadResultSetDataSource pid=55238) /Users/shchen/snowflake/mlruntimes/env_service/lib/python3.11/site-packages/snowflake/connector/config_manager.py:351: UserWarning: Bad owner or permissions on /Users/shchen/.snowflake/connections.toml.

(ReadResultSetDataSource pid=55238)  * To change owner, run `chown $USER "/Users/shchen/.snowflake/connections.toml"`.
(ReadResultSetDataSource pid=55238)  * To restrict permissions, run `chmod 0600 "/Users/shchen/.snowflake/connections.toml"`.

(ReadResultSetDataSource pid=55238) 
(ReadResultSetDataSource pid=55238)   warn(f"Bad owner or permissions on {str(filep)}{chmod_message}")

(ReadResultSetDataSource pid=55237) /Users/shchen/snowflake/mlruntimes/env_service/lib/python3.11/site-packages/snowflake/connector/config_manager.py:351: UserWarning: Bad owner or permissions on /Users/shchen/.

In [10]:
# In this example, each parameter has 2 possible values, so the total number of unique combinations is 2 × 2 × 2 = 8.
tuner_results.results

,accuracy,should_checkpoint,trial_id,time_total_s,config/learning_rate,config/max_depth,config/n_estimators
0,0.967989,True,34fac_00000,5.755174,0.01,4,50
1,0.983299,True,34fac_00001,4.344136,0.03,4,50
2,0.981211,True,34fac_00002,4.127693,0.01,5,50
3,0.995129,True,34fac_00003,3.705020,0.03,5,50
4,0.967989,True,34fac_00004,3.517283,0.01,4,51
5,0.984690,True,34fac_00005,3.307985,0.03,4,51
6,0.982603,True,34fac_00006,3.603007,0.01,5,51
7,0.995825,True,34fac_00007,3.585153,0.03,5,51
